In [ ]:
!git lfs initialized
!git clone https://huggingface.co/OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash

!yum install git-lfs

!git lfs install


In [ ]:
pip install transformers

In [1]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("./oasst-sft-4-pythia-12b-epoch-3.5")

model = AutoModelForCausalLM.from_pretrained("./oasst-sft-4-pythia-12b-epoch-3.5")
model.half()
model = model.cuda()
print("model loaded.")


def infer(prompt:str):
    prompt = f'<|prompter|>{prompt}<|endoftext|><|assistant|>'
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output_ids = model.generate(input_ids,
                                    max_length=100,
                                    num_beams=1,
                                    temperature=0.9,
                                    top_p=0.9)
    response = tokenizer.decode(output_ids[0].cpu(), skip_special_tokens=True)
    return response

if __name__ == '__main__':
    # 注意，由于显存约束，模型能处理的长度有限，这里每次只是单轮对话。
    # 多轮对话的格式
    #   <|prompter|>some text<|endoftext|><|assistant|>some response<|endoftext|><|prompter|>some text...
    # 注意到tokenizer的speical_tokens里有<|system|>，猜想功能应该和ChatGPT差不多，不过我没有测
    
    
    prompt = "您好"
    
    print("done")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model loaded.
done


In [ ]:
import json
def do_generation():
    
    with open('data/generation/dataclean.json','r') as f:
        data_all=json.load(f)
    ans=[];
    path='./generation_gpt_saved.json'
    right=0;
    wrong=0
    for i in data_all:
        
        s=i['question']
        tmp = infer(s)
        print(tmp)
        ans.append(tmp)
        
#         print("正确答案是"+i['groundTruth'][0])
#         if(i['groundTruth'][0] in tmp):
#             right=right+1
#             print("此题正确")
#         else:
#             wrong=wrong+1
#             print("此题错误")
#     print(right*1.0/(right+wrong))
#     print(right+wrong)
    with open(path,'w') as f:
        json.dump(ans,f);
        
do_generation();

In [ ]:
!unzip data.zip